In [96]:
import pandas as pd
import glob
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
%matplotlib inline



In [97]:
#path to celllines folder where I have 44 celllines
path = './celllines' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    print(filename)
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

./celllines/HCC3153.csv
./celllines/HCC38.csv
./celllines/HCC1419.csv
./celllines/MDAMB175VII.csv
./celllines/HCC1395.csv
./celllines/BT549.csv
./celllines/HCC1187.csv
./celllines/CAL851.csv
./celllines/HCC1569.csv
./celllines/MDAMB157.csv
./celllines/DU4475.csv
./celllines/BT474.csv
./celllines/CAL148.csv
./celllines/184A1.csv
./celllines/MFM223.csv
./celllines/HBL100.csv
./celllines/HCC70.csv
./celllines/UACC812.csv
./celllines/MDAMB453.csv
./celllines/OCUBM.csv
./celllines/BT20.csv
./celllines/CAL51.csv
./celllines/MDAkb2.csv
./celllines/EVSAT.csv
./celllines/MDAMB134VI.csv
./celllines/MX1.csv
./celllines/MPE600.csv
./celllines/HCC2185.csv
./celllines/HCC1937.csv
./celllines/HCC2157.csv
./celllines/JIMT1.csv
./celllines/HCC1500.csv
./celllines/HDQP1.csv
./celllines/MCF10A.csv
./celllines/HCC1599.csv
./celllines/MDAMB415.csv
./celllines/T47D.csv
./celllines/HCC1954.csv
./celllines/MCF10F.csv
./celllines/EFM192A.csv
./celllines/MCF7.csv
./celllines/MDAMB361.csv


In [98]:
len(frame)

16320112

In [99]:
frame.isnull().values.any()

True

In [100]:
df=frame.dropna()

In [101]:
len(df)

10563767

In [102]:
frame.shape

(16320112, 42)

In [103]:
df.shape

(10563767, 42)

In [104]:
df.reset_index(drop=True, inplace=True)

In [105]:
df.head()

,treatment,cell_line,time,cellID,fileID,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,...,p.PDPK1,p.PLCg2,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,EGF,HCC3153,0.0,1,1090,1.227645,0.898342,2.00988,2.355446,7.47908,...,1.976894,2.917418,2.34416,4.88364,2.364620,2.103391,2.855296,2.841560,1.812054,3.059652
1,EGF,HCC3153,0.0,1,1174,0.224784,1.406103,1.20489,1.812234,4.25020,...,1.429694,1.541972,2.20794,4.95688,1.584953,0.551474,2.194431,1.931195,1.323898,2.248363
2,EGF,HCC3153,0.0,2,1090,0.224784,1.480403,2.82043,0.909058,11.79636,...,1.581918,1.724207,6.35139,6.26024,1.822729,0.871634,2.149061,2.520615,1.218312,3.453422
3,EGF,HCC3153,0.0,2,1174,1.478311,2.468120,1.20489,3.084831,4.61839,...,1.683564,0.986227,3.64678,7.13692,1.123174,0.551474,2.134567,1.942359,0.824224,2.065399
4,EGF,HCC3153,0.0,3,1090,1.578879,3.019824,1.22424,2.833176,5.64244,...,0.983155,0.986227,4.78460,3.65731,0.592434,2.210619,1.722814,1.860546,2.422809,1.872065


In [106]:
df.drop(['treatment','cell_line','time','cellID','fileID'], axis=1,inplace=True)

/Users/albarasa/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [107]:

df.head()

,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,p.Akt.Ser473.,p.AKT.Thr308.,p.AMPK,...,p.PDPK1,p.PLCg2,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,1.227645,0.898342,2.00988,2.355446,7.47908,3.90310,3.746233,3.266905,2.015977,2.942359,...,1.976894,2.917418,2.34416,4.88364,2.364620,2.103391,2.855296,2.841560,1.812054,3.059652
1,0.224784,1.406103,1.20489,1.812234,4.25020,2.26248,1.214976,4.242500,2.210044,1.706085,...,1.429694,1.541972,2.20794,4.95688,1.584953,0.551474,2.194431,1.931195,1.323898,2.248363
2,0.224784,1.480403,2.82043,0.909058,11.79636,3.35331,2.152767,2.990132,0.895138,1.966050,...,1.581918,1.724207,6.35139,6.26024,1.822729,0.871634,2.149061,2.520615,1.218312,3.453422
3,1.478311,2.468120,1.20489,3.084831,4.61839,1.59724,0.993163,4.222791,2.445925,2.499050,...,1.683564,0.986227,3.64678,7.13692,1.123174,0.551474,2.134567,1.942359,0.824224,2.065399
4,1.578879,3.019824,1.22424,2.833176,5.64244,2.30952,1.828138,1.796079,3.347871,2.620279,...,0.983155,0.986227,4.78460,3.65731,0.592434,2.210619,1.722814,1.860546,2.422809,1.872065


In [108]:
#p.ERK	p.Akt.Ser473.	p.S6	p.HER2	p.PLCg2
target1=df['p.ERK']
target2=df['p.Akt.Ser473.']
target3=df['p.S6']
target4=df['p.HER2']
target5=df['p.PLCg2']



In [109]:
df.drop(['p.ERK', 'p.Akt.Ser473.','p.S6','p.HER2','p.PLCg2'], axis=1,inplace=True)

/Users/albarasa/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [110]:
target1.head()

0    3.869078
1    2.818969
2    2.852269
3    2.417179
4    2.913117
Name: p.ERK, dtype: float64

In [111]:
df.head()

,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,p.4EBP1,p.AKT.Thr308.,p.AMPK,p.BTK,...,p.p53,p.p90RSK,p.PDPK1,p.RB,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,1.227645,0.898342,2.00988,2.355446,7.47908,3.90310,3.746233,2.015977,2.942359,3.251386,...,1.444603,4.71609,1.976894,2.34416,2.364620,2.103391,2.855296,2.841560,1.812054,3.059652
1,0.224784,1.406103,1.20489,1.812234,4.25020,2.26248,1.214976,2.210044,1.706085,2.364511,...,0.879170,4.33499,1.429694,2.20794,1.584953,0.551474,2.194431,1.931195,1.323898,2.248363
2,0.224784,1.480403,2.82043,0.909058,11.79636,3.35331,2.152767,0.895138,1.966050,2.481327,...,0.886507,2.85946,1.581918,6.35139,1.822729,0.871634,2.149061,2.520615,1.218312,3.453422
3,1.478311,2.468120,1.20489,3.084831,4.61839,1.59724,0.993163,2.445925,2.499050,2.400367,...,0.798554,3.33597,1.683564,3.64678,1.123174,0.551474,2.134567,1.942359,0.824224,2.065399
4,1.578879,3.019824,1.22424,2.833176,5.64244,2.30952,1.828138,3.347871,2.620279,3.597402,...,2.039896,3.69760,0.983155,4.78460,0.592434,2.210619,1.722814,1.860546,2.422809,1.872065


In [112]:
X = df
y = target1
model1 = sm.OLS(y, X).fit()
predictions = model1.predict(X)
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  p.ERK   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                 5.964e+06
Date:                Thu, 26 Sep 2019   Prob (F-statistic):               0.00
Time:                        11:35:31   Log-Likelihood:            -1.1592e+07
No. Observations:            10563767   AIC:                         2.318e+07
Df Residuals:                10563735   BIC:                         2.318e+07
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
b.CATENIN      3.754e-05      0.000      0.087      0.931      -0.001       0.001
cleavedCas        0.0044      0.000     17.066      0.000       0.004       0.005
CyclinB          -0.0326      0.000   -138.196      0.000      -0.033      -0.032
GAPDH             0.0708      0.000    226.514      0.000       0.070       0.071
IdU               0.0043   9.69e-05     44.311      0.000       0.004       0.004
Ki.67            -0.0361      0.000   -178.922      0.000      -0.037      -0.036
p.4EBP1           0.0225      0.000     85.878      0.000       0.022       0.023
p.AKT.Thr308.     0.0514      0.000    163.613      0.000       0.051       0.052
p.AMPK           -0.0047      0.000    -11.777      0.000      -0.006      -0.004
p.BTK            -0.0933      0.000   -256.460      0.000      -0.094      -0.093
p.CREB            0.0263      0.000     69.326      0.000       0.026       0.027
p.FAK             0.0270      0.000     79.347      0.000       0.026       0.028
p.GSK3b          -0.0717      0.000   -256.984      0.000      -0.072      -0.071
p.H3             -0.0711      0.000   -260.337      0.000      -0.072      -0.071
p.JNK            -0.0447      0.000   -132.631      0.000      -0.045      -0.044
p.MAP2K3          0.0224      0.000     59.755      0.000       0.022       0.023
p.MAPKAPK2        0.0889      0.000    249.349      0.000       0.088       0.090
p.MEK             0.0306      0.000     74.062      0.000       0.030       0.031
p.MKK3.MKK6      -0.1028      0.000   -320.751      0.000      -0.103      -0.102
p.MKK4            0.2346      0.000    768.387      0.000       0.234       0.235
p.NFkB            0.0580      0.001    109.462      0.000       0.057       0.059
p.p38             0.0428      0.000    143.944      0.000       0.042       0.043
p.p53            -0.0374      0.000   -150.703      0.000      -0.038      -0.037
p.p90RSK          0.4195      0.000   1587.144      0.000       0.419       0.420
p.PDPK1          -0.0224      0.000    -67.412      0.000      -0.023      -0.022
p.RB             -0.0040      0.000    -24.237      0.000      -0.004      -0.004
p.S6K             0.0792      0.000    206.492      0.000       0.078       0.080
p.SMAD23          0.0486      0.000    142.969      0.000       0.048       0.049
p.SRC             0.0318      0.000     94.316      0.000       0.031       0.032
p.STAT1           0.1099      0.000    248.790      0.000       0.109       0.111
p.STAT3           0.0939      0.000    423.994      0.000       0.093       0.094
p.STAT5           0.0011      0.000      2.758      0.006       0.000       0.002
==============================================================================
Omnibus:                   508075.718   Durbin-Watson:                   1.401
Prob(Omnibus):                  0.000   Jarque-Bera (JB

In [113]:
#Test
# Read data from file '.csv' 
temp_data = pd.read_csv("subchallenge_1_template_data.csv",encoding='latin-1',delimiter=',' ) 

In [114]:
temp_data.head()

,glob_cellID,cell_line,treatment,time,cellID,fileID,p.ERK,p.Akt.Ser473.,p.S6,p.HER2,p.PLCg2
0,1,AU565,EGF,0.0,1,59,NaN,NaN,NaN,NaN,NaN
1,2,AU565,EGF,0.0,1,122,NaN,NaN,NaN,NaN,NaN
2,3,AU565,EGF,0.0,2,59,NaN,NaN,NaN,NaN,NaN
3,4,AU565,EGF,0.0,2,122,NaN,NaN,NaN,NaN,NaN
4,5,AU565,EGF,0.0,3,59,NaN,NaN,NaN,NaN,NaN


In [115]:
len(temp_data)

2383058

In [140]:
#path to subchallenge folder where I have 6 celllines
path = './sub1' # use your path
all_files = [path + "/AU565.csv",path + "/EFM19.csv", path + "/HCC2218.csv",
             path + "/LY2.csv",path + "/MACLS2.csv",  path + "/MDAMB436.csv"]

li = []

for filename in all_files:
    print(filename)
    d = pd.read_csv(filename, index_col=None, header=0)
    li.append(d)

frame_test = pd.concat(li, axis=0, ignore_index=True)


./sub1/AU565.csv
./sub1/EFM19.csv
./sub1/HCC2218.csv
./sub1/LY2.csv
./sub1/MACLS2.csv
./sub1/MDAMB436.csv


In [117]:
frame_test.drop(['p.ERK', 'p.Akt.Ser473.','p.S6','p.HER2','p.PLCg2'], axis=1,inplace=True)

In [118]:
frame_test.drop(['treatment','cell_line','time','cellID','fileID'], axis=1,inplace=True)

In [119]:
frame_test.shape

(2383058, 32)

In [120]:
target1_pred = model1.predict(frame_test)

In [121]:
len(target1_pred)

2383058

In [122]:
X = df
y = target2
model2 = sm.OLS(y, X).fit()
predictions = model2.predict(X)
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p.Akt.Ser473.   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                 7.092e+06
Date:                Thu, 26 Sep 2019   Prob (F-statistic):               0.00
Time:                        11:38:19   Log-Likelihood:            -1.2819e+07
No. Observations:            10563767   AIC:                         2.564e+07
Df Residuals:                10563735   BIC:                         2.564e+07
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
b.CATENIN        -0.0087      0.000    -17.870      0.000      -0.010      -0.008
cleavedCas       -0.0795      0.000   -274.531      0.000      -0.080      -0.079
CyclinB          -0.0154      0.000    -58.031      0.000      -0.016      -0.015
GAPDH            -0.0018      0.000     -5.043      0.000      -0.002      -0.001
IdU               0.0642      0.000    589.883      0.000       0.064       0.064
Ki.67            -0.0167      0.000    -73.568      0.000      -0.017      -0.016
p.4EBP1           0.0515      0.000    174.919      0.000       0.051       0.052
p.AKT.Thr308.     0.1182      0.000    334.667      0.000       0.117       0.119
p.AMPK            0.0296      0.000     65.455      0.000       0.029       0.030
p.BTK            -0.1141      0.000   -279.272      0.000      -0.115      -0.113
p.CREB            0.0478      0.000    112.152      0.000       0.047       0.049
p.FAK            -0.0428      0.000   -111.952      0.000      -0.044      -0.042
p.GSK3b           0.4176      0.000   1332.439      0.000       0.417       0.418
p.H3             -0.0091      0.000    -29.639      0.000      -0.010      -0.008
p.JNK            -0.0877      0.000   -231.576      0.000      -0.088      -0.087
p.MAP2K3         -0.0325      0.000    -77.176      0.000      -0.033      -0.032
p.MAPKAPK2        0.0780      0.000    194.915      0.000       0.077       0.079
p.MEK             0.1151      0.000    247.890      0.000       0.114       0.116
p.MKK3.MKK6       0.0030      0.000      8.432      0.000       0.002       0.004
p.MKK4           -0.0047      0.000    -13.724      0.000      -0.005      -0.004
p.NFkB            0.1675      0.001    281.511      0.000       0.166       0.169
p.p38             0.1943      0.000    581.215      0.000       0.194       0.195
p.p53             0.1313      0.000    471.608      0.000       0.131       0.132
p.p90RSK          0.0309      0.000    103.990      0.000       0.030       0.031
p.PDPK1          -0.0716      0.000   -192.134      0.000      -0.072      -0.071
p.RB             -0.0479      0.000   -256.591      0.000      -0.048      -0.048
p.S6K             0.1354      0.000    314.548      0.000       0.135       0.136
p.SMAD23         -0.0395      0.000   -103.418      0.000      -0.040      -0.039
p.SRC             0.0281      0.000     74.211      0.000       0.027       0.029
p.STAT1           0.0256      0.000     51.613      0.000       0.025       0.027
p.STAT3           0.0468      0.000    188.327      0.000       0.046       0.047
p.STAT5           0.1134      0.000    250.302      0.000       0.113       0.114
==============================================================================
Omnibus:                   135340.633   Durbin-Watson:                   1.225
Prob(Omnibus):                  0.000   Jarque-Bera (JB

In [123]:
target2_pred = model2.predict(frame_test)

In [124]:
len(target2_pred)

2383058

In [125]:
X = df
y = target3
model3 = sm.OLS(y, X).fit()
predictions = model3.predict(X)
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   p.S6   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 7.286e+06
Date:                Thu, 26 Sep 2019   Prob (F-statistic):               0.00
Time:                        11:39:51   Log-Likelihood:            -1.7030e+07
No. Observations:            10563767   AIC:                         3.406e+07
Df Residuals:                10563735   BIC:                         3.406e+07
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
b.CATENIN        -0.0924      0.001   -127.509      0.000      -0.094      -0.091
cleavedCas        0.0464      0.000    107.659      0.000       0.046       0.047
CyclinB          -0.0717      0.000   -181.792      0.000      -0.073      -0.071
GAPDH             0.1430      0.001    273.251      0.000       0.142       0.144
IdU               0.1137      0.000    701.088      0.000       0.113       0.114
Ki.67             0.0109      0.000     32.400      0.000       0.010       0.012
p.4EBP1          -0.1286      0.000   -293.135      0.000      -0.129      -0.128
p.AKT.Thr308.    -0.0410      0.001    -77.977      0.000      -0.042      -0.040
p.AMPK            0.0151      0.001     22.355      0.000       0.014       0.016
p.BTK            -0.1429      0.001   -234.721      0.000      -0.144      -0.142
p.CREB            0.0362      0.001     57.068      0.000       0.035       0.037
p.FAK            -0.0149      0.001    -26.197      0.000      -0.016      -0.014
p.GSK3b           0.4484      0.000    960.409      0.000       0.447       0.449
p.H3              0.2823      0.000    617.420      0.000       0.281       0.283
p.JNK             0.0079      0.001     14.064      0.000       0.007       0.009
p.MAP2K3         -0.0822      0.001   -131.232      0.000      -0.083      -0.081
p.MAPKAPK2        0.0624      0.001    104.575      0.000       0.061       0.064
p.MEK            -0.0112      0.001    -16.238      0.000      -0.013      -0.010
p.MKK3.MKK6      -0.1316      0.001   -245.452      0.000      -0.133      -0.131
p.MKK4            0.1148      0.001    224.694      0.000       0.114       0.116
p.NFkB            0.2819      0.001    317.956      0.000       0.280       0.284
p.p38             0.0019      0.000      3.871      0.000       0.001       0.003
p.p53            -0.0126      0.000    -30.471      0.000      -0.013      -0.012
p.p90RSK          0.1309      0.000    295.969      0.000       0.130       0.132
p.PDPK1           0.0677      0.001    121.918      0.000       0.067       0.069
p.RB              0.0973      0.000    349.584      0.000       0.097       0.098
p.S6K            -0.0447      0.001    -69.760      0.000      -0.046      -0.043
p.SMAD23         -0.0284      0.001    -49.882      0.000      -0.030      -0.027
p.SRC             0.0210      0.001     37.298      0.000       0.020       0.022
p.STAT1           0.3739      0.001    505.866      0.000       0.372       0.375
p.STAT3           0.0487      0.000    131.398      0.000       0.048       0.049
p.STAT5           0.0700      0.001    103.605      0.000       0.069       0.071
==============================================================================
Omnibus:                   221434.080   Durbin-Watson:                   1.298
Prob(Omnibus):                  0.000   Jarque-Bera (JB

In [126]:
target3_pred = model3.predict(frame_test)

In [127]:
X = df
y = target4
model4 = sm.OLS(y, X).fit()
predictions = model4.predict(X)
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 p.HER2   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                 1.785e+07
Date:                Thu, 26 Sep 2019   Prob (F-statistic):               0.00
Time:                        11:41:20   Log-Likelihood:            -9.6726e+06
No. Observations:            10563767   AIC:                         1.935e+07
Df Residuals:                10563735   BIC:                         1.935e+07
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
b.CATENIN         0.0512      0.000    141.676      0.000       0.050       0.052
cleavedCas        0.0644      0.000    299.332      0.000       0.064       0.065
CyclinB          -0.0509      0.000   -258.585      0.000      -0.051      -0.050
GAPDH             0.0503      0.000    192.856      0.000       0.050       0.051
IdU               0.0687   8.08e-05    849.312      0.000       0.068       0.069
Ki.67             0.0003      0.000      1.671      0.095   -4.86e-05       0.001
p.4EBP1          -0.0293      0.000   -134.248      0.000      -0.030      -0.029
p.AKT.Thr308.     0.1159      0.000    441.980      0.000       0.115       0.116
p.AMPK            0.1710      0.000    509.622      0.000       0.170       0.172
p.BTK             0.2215      0.000    730.187      0.000       0.221       0.222
p.CREB           -0.0291      0.000    -92.075      0.000      -0.030      -0.028
p.FAK             0.1287      0.000    453.865      0.000       0.128       0.129
p.GSK3b           0.0262      0.000    112.454      0.000       0.026       0.027
p.H3              0.0408      0.000    178.948      0.000       0.040       0.041
p.JNK             0.0654      0.000    232.675      0.000       0.065       0.066
p.MAP2K3          0.0016      0.000      5.136      0.000       0.001       0.002
p.MAPKAPK2       -0.0412      0.000   -138.626      0.000      -0.042      -0.041
p.MEK             0.0243      0.000     70.498      0.000       0.024       0.025
p.MKK3.MKK6      -0.0723      0.000   -270.366      0.000      -0.073      -0.072
p.MKK4            0.1145      0.000    449.559      0.000       0.114       0.115
p.NFkB            0.0393      0.000     89.040      0.000       0.038       0.040
p.p38             0.0160      0.000     64.314      0.000       0.015       0.016
p.p53            -0.0313      0.000   -151.296      0.000      -0.032      -0.031
p.p90RSK          0.0601      0.000    272.888      0.000       0.060       0.061
p.PDPK1           0.0178      0.000     64.371      0.000       0.017       0.018
p.RB              0.0563      0.000    406.049      0.000       0.056       0.057
p.S6K            -0.0485      0.000   -151.701      0.000      -0.049      -0.048
p.SMAD23          0.0502      0.000    176.897      0.000       0.050       0.051
p.SRC             0.1439      0.000    512.516      0.000       0.143       0.144
p.STAT1          -0.0013      0.000     -3.436      0.001      -0.002      -0.001
p.STAT3           0.0657      0.000    355.758      0.000       0.065       0.066
p.STAT5           0.2443      0.000    726.108      0.000       0.244       0.245
==============================================================================
Omnibus:                    95918.386   Durbin-Watson:                   1.481
Prob(Omnibus):                  0.000   Jarque-Bera (JB

In [128]:
target4_pred = model4.predict(frame_test)

In [129]:
X = df
y = target5
model5 = sm.OLS(y, X).fit()
predictions = model5.predict(X)
model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                p.PLCg2   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                 3.943e+06
Date:                Thu, 26 Sep 2019   Prob (F-statistic):               0.00
Time:                        11:42:43   Log-Likelihood:            -1.2250e+07
No. Observations:            10563767   AIC:                         2.450e+07
Df Residuals:                10563735   BIC:                         2.450e+07
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
b.CATENIN         0.0067      0.000     14.458      0.000       0.006       0.008
cleavedCas       -0.0991      0.000   -361.111      0.000      -0.100      -0.099
CyclinB          -0.0048      0.000    -19.006      0.000      -0.005      -0.004
GAPDH            -0.0113      0.000    -34.001      0.000      -0.012      -0.011
IdU               0.0051      0.000     49.713      0.000       0.005       0.005
Ki.67             0.0479      0.000    223.100      0.000       0.048       0.048
p.4EBP1          -0.0204      0.000    -72.946      0.000      -0.021      -0.020
p.AKT.Thr308.    -0.0798      0.000   -238.448      0.000      -0.080      -0.079
p.AMPK           -0.1306      0.000   -304.946      0.000      -0.131      -0.130
p.BTK             0.2947      0.000    761.132      0.000       0.294       0.295
p.CREB            0.0520      0.000    128.956      0.000       0.051       0.053
p.FAK             0.3400      0.000    939.662      0.000       0.339       0.341
p.GSK3b           0.2030      0.000    683.769      0.000       0.202       0.204
p.H3              0.0324      0.000    111.466      0.000       0.032       0.033
p.JNK             0.0198      0.000     55.198      0.000       0.019       0.021
p.MAP2K3          0.0014      0.000      3.536      0.000       0.001       0.002
p.MAPKAPK2       -0.0141      0.000    -37.057      0.000      -0.015      -0.013
p.MEK            -0.1551      0.000   -352.619      0.000      -0.156      -0.154
p.MKK3.MKK6       0.0966      0.000    283.243      0.000       0.096       0.097
p.MKK4            0.0679      0.000    208.990      0.000       0.067       0.069
p.NFkB           -0.1425      0.001   -252.773      0.000      -0.144      -0.141
p.p38             0.0411      0.000    129.631      0.000       0.040       0.042
p.p53             0.0035      0.000     13.341      0.000       0.003       0.004
p.p90RSK          0.1648      0.000    585.968      0.000       0.164       0.165
p.PDPK1           0.0108      0.000     30.722      0.000       0.010       0.012
p.RB             -0.0113      0.000    -63.924      0.000      -0.012      -0.011
p.S6K            -0.0510      0.000   -124.988      0.000      -0.052      -0.050
p.SMAD23         -0.0296      0.000    -81.770      0.000      -0.030      -0.029
p.SRC             0.2570      0.000    717.089      0.000       0.256       0.258
p.STAT1          -0.1131      0.000   -240.477      0.000      -0.114      -0.112
p.STAT3           0.0423      0.000    179.493      0.000       0.042       0.043
p.STAT5           0.1824      0.000    424.623      0.000       0.182       0.183
==============================================================================
Omnibus:                     5627.538   Durbin-Watson:                   1.653
Prob(Omnibus):                  0.000   Jarque-Bera (JB

In [130]:
target5_pred = model5.predict(frame_test)

In [131]:
#glob_cellID	cell_line	treatment	time	cellID	fileID	p.ERK	p.Akt.Ser473.	p.S6	p.HER2	p.PLCg2
temp_data['p.ERK']=target1_pred 

In [132]:
temp_data['p.Akt.Ser473.']=target2_pred 

In [133]:
temp_data['p.S6']=target3_pred 

In [134]:
temp_data['p.HER2']=target4_pred 

In [135]:
temp_data['p.PLCg2']=target5_pred 

In [136]:
export_csv = temp_data.to_csv (r'subchallenge_1.csv', index = None, header=True)

In [137]:
temp_data.head()

,glob_cellID,cell_line,treatment,time,cellID,fileID,p.ERK,p.Akt.Ser473.,p.S6,p.HER2,p.PLCg2
0,1,AU565,EGF,0.0,1,59,3.382208,4.675948,5.750634,4.048802,2.473475
1,2,AU565,EGF,0.0,1,122,3.648685,4.423118,7.041247,5.251044,2.797731
2,3,AU565,EGF,0.0,2,59,3.732280,4.525201,6.308761,5.018964,2.699051
3,4,AU565,EGF,0.0,2,122,2.869946,3.739948,5.521329,4.360088,2.001269
4,5,AU565,EGF,0.0,3,59,2.185578,2.497834,3.386253,3.507900,1.452643


In [138]:
temp_data.tail()

,glob_cellID,cell_line,treatment,time,cellID,fileID,p.ERK,p.Akt.Ser473.,p.S6,p.HER2,p.PLCg2
2383053,2383054,MDAMB436,iPKC,40.0,3563,2108,4.060104,3.863664,6.756930,4.849722,2.996999
2383054,2383055,MDAMB436,iPKC,40.0,3564,2108,4.126342,3.836747,6.288548,4.777534,3.114046
2383055,2383056,MDAMB436,iPKC,40.0,3565,2108,3.505406,4.105756,7.271371,4.856906,2.796211
2383056,2383057,MDAMB436,iPKC,40.0,3566,2108,2.417025,3.148542,4.363266,2.932465,1.519235
2383057,2383058,MDAMB436,iPKC,40.0,3567,2108,3.976391,3.823848,5.844144,4.279650,2.045471
